In [ ]:
!pip install pandas scikit-learn langchain_core langchain_community langchain_openai langchain_text_splitters PyPDF2 faiss-cpu pymupdf --quiet

In [ ]:
# LIBRARIES #
import os
import io
import requests
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import fitz
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from PyPDF2 import PdfReader
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import tqdm
import faiss
import re

In [ ]:
# LLM #
os.environ["OPENAI_API_KEY"] = ""

print("OPENAI_API_KEY environment variable set.")

llm = ChatOpenAI(
    model="o3-mini",
    api_key=os.getenv("OPENAI_API_KEY")
)

OPENAI_API_KEY environment variable set.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# READING PDFs #
# Dictionary of PDF names and paths
pdf_paths = {
    "KXE_brochure": "/content/drive/My Drive/Colab Notebooks/998KXE_Brochure.pdf",
    "XE_brochure": "/content/drive/My Drive/Colab Notebooks/998XE_Brochure.pdf",
    "XE_specs": "/content/drive/My Drive/Colab Notebooks/998XE_Specifications.pdf",
    "risk_assessment": "/content/drive/My Drive/Colab Notebooks/Risk_Assessment.pdf"
}

# Extract text from PDFs
def extract_text(pdf_path):
    doc = fitz.open(pdf_path) # Open the PDF document
    text = ""
    for page in doc:
        text += page.get_text() # Extract text from each page
    return text
pdf_texts = {label: extract_text(path) for label, path in pdf_paths.items()}

# Split texts into chunks with labels
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts, labels = [], []

for label, text in pdf_texts.items():
    chunks = splitter.split_text(text)
    texts.extend(chunks)
    labels.extend([label] * len(chunks))

# Create embeddings
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")
X = [embeddings_model.embed_query(t) for t in tqdm.tqdm(texts, desc="Embedding PDF chunks")]
X = np.array(X)
y = np.array(labels)

# Supervised Classifier
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X, y)

# Build FAISS vector stores for each PDF
vectorstores = {}
for label, text in pdf_texts.items():
    chunks = splitter.split_text(text)
    vectorstores[label] = FAISS.from_texts(chunks, embeddings_model)

# labels - KXE_brochure, XE_brochure, XE_specs, risk_assessment

Embedding PDF chunks: 100%|██████████| 278/278 [01:02<00:00,  4.45it/s]


In [ ]:
# CSV FILES #
non_generic_parts_path = "/content/drive/My Drive/Colab Notebooks/50_Spare_Parts.csv"
non_generic_parts = pd.read_csv(non_generic_parts_path)

failure_path = "/content/drive/My Drive/Colab Notebooks/Failure_Data.csv"
failures = pd.read_csv(failure_path, encoding = "cp1252")

original_critical_list_path = "/content/drive/My Drive/Colab Notebooks/Original_Critical_Spare_Parts_List.csv"
original_critical_spares_list = pd.read_csv(original_critical_list_path)

# Convert each part to a string
parts_list = non_generic_parts.apply(
    lambda row: f"{row['CAT Part']}: {row['SAP Material Description']}: {row['U801 MRP']}: {row['Unit Cost']}: {row['Lead Time']}", axis = 1).tolist()

failure_list = failures.apply(
    lambda row: f"{row['Order']}: {row['Notification']}: {row['Order Type']}: {row['Order Long Text Description']}: {row['Notification Long Text Description']}: {row['Sort field']}: {row['Total Costs']}: {row['Total Work Hours ']}", axis = 1).tolist()
all_failure_text = "\n".join(failure_list)

critical_list = original_critical_spares_list.apply(
    lambda row: f"{row['Material Number']}: {row['Description']}: {row['Failure mode']}: {row['Maintenance Strategy']}: {row['Expected lead time']}: {row['Is a repairable?']}: {row['Highest Potential Impact']}: {row['Highest Potential Impact Cost']}: {row['Likelihood']}: {row['Estimated annual usage']}: {row['Usage frequency']}: {row['Justification commentary']}", axis = 1).tolist()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
failure_chunks = text_splitter.split_text(all_failure_text)
embeddings = OpenAIEmbeddings()
failures_vectorstore = FAISS.from_texts(failure_chunks, embeddings)
failures_retriever = failures_vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})


In [ ]:
# PROMPTING #
prompt = ChatPromptTemplate.from_template(
"""
STEP 1
You are acting as a reliability engineer with expertise in mechanical engineering and the mining industry.
Your task is to analyse historical failure data for two Caterpillar (CAT) 998KXE front-end loaders at the North Queensland Hard
Rock Mine (NQHRM). Your goal is to produce a critical spare parts list for the 998KXE front-end loaders to minimise operational downtime
and financial losses.

STEP 2
I have provided the official 998KXE brochure and the brochure/specifications of the older but similar 998XE model, which has more
publicly available data online. Use both documents, plus any additional knowledge you have of CAT equipment to make well-informed decisions
about spare parts:

{KXE_brochure} {XE_brochure} {XE_specs}

STEP 3
A critical spares list identifies the essential parts that must be
stocked on-site to minimise machine downtime. Parts are considered critical if:
•       They have long lead times OR
•       They are linked to high financial losses from machine downtime OR
•       They contribute to unplanned downtime when they fail

The critical spare parts list excludes parts already stocked on site (V1 classified parts) and generic parts such as nuts and bolts.
Currently, the NQHRM is producing their critical spare parts lists manually and I have provided a sample of their list for the 998KXE
front-end loader:

{original_critical_spares_list}

STEP 4
You have been provided with two Excel files which you will use to generate the critical spare parts list:
Non-Generic Spare Parts: {spare_parts}
•       CAT Part: 7-digit part number from CAT. (Part number = CAT part = CAT part number)
•       7-digit number (eg: 489-1166) OR
•       6-digit number with the second digit being a letter (eg: 8T-9527)
•       SAP Material Description: Short description of the CAT part number
•       U801 MRP:
•           ND: Marked for deletion (can be changed)
•           PD: Catalogued but not stocked
•           V1: Already stocked on-site
•       Unit Cost: Part cost
•       Lead Time: Expected delivery time from CAT or supplier to the mine

Downtime History Data: {failures}
•       Order number: 10-digit number corresponding to the order (linked to notification)
•       Notification number: 10-digit number corresponding to the notification (linked to order)
•       Order type:
•           PM01: Corrective maintenance identified during scheduled servicing (non-urgent)
•           PM02: Scheduled preventive maintenance (mostly labour)
•           PM03: Emergency corrective maintenance for unexpected failures (immediate)
•       Order long text description: Maintenance description
•           Eg: “4W Mech Svce Bucket CAT 988KXE Loader 1” which means there is scheduled maintenance every four weeks on the CAT 988KXE Loader 1 bucket.
•       Notification long text description: Failure details including timestamp, repair details and sometimes the CAT part number
•       Sort field: Machine identifier (Loader 1 or Loader 2)
•       Total costs: Cost of repairing/replacing the part inclusive of labour hire
•       Total work hours: Downtime duration

If data is missing, use 998XKE failure rates or industry benchmarks (eg: from CATs website) to fill in the gaps and document assumptions.

While the two datasets are not directly linked, part matches can be made by:
•       Comparing the notification long text with CAT part numbers
•       Comparing the order long text with the SAP material description
•       Applying domain knowledge of 998KXE/998XE systems


STEP 5
Criticality score = Likelihood score x Severity score
•       Likelihood score: Frequency of failure occurrences. Calculate the failure count for PM03 records in the downtime history data.
    Exclude PM01 and PM02 unless explicitly ties to a failure
•       Severity Score: The financial impact of machine downtime. Calculate as the sum of unit cost (from non-generic parts list),
    labour cost (from downtime history) and downtime cost ($10,400 per day x total work hours converted to days)

Use the likelihood and severity risk tables provided to identify the scores of each spare part.
Then use the provided risk assessment matrix to calculate the criticality score. For example, a low severity and unlikely
likelihood results in a risk score of 0.3:

{risk_assessment}

Any part with a criticality score greater than 30 is classified as a critical spare part. Therefore, generate a critical spare parts list for the 998KXE front end-loader that follows the below structure
•       CAT Part
•       SAP Material Description
•       U801 MRP
•       Unit Cost (USD)
•       Lead Time (days)
•       Assumed Likelihood Score
•       Estimated Severity Score
•       Criticality Score
•       Comments

STEP 6
Before finalising the critical spare parts list, verify:
•       All risk scores greater than 30 are included
•       V1 classified parts are excluded
•       Calculations align with provided matrices
•       Part numbers are valid  CAT numbers

""")

In [ ]:
## RETRIEVAL ##
# Process each spare part
critical_spares_list = []
embedding_cache = {}

def get_cached_embedding(text):
    if text not in embedding_cache:
        embedding_cache[text] = embeddings_model.embed_query(text)
    return embedding_cache[text]

for part in parts_list:
    part_emb = get_cached_embedding(part)
    predicted_label = clf.predict([part_emb])[0]

    # Retrieve top 5 relevant chunks from predicted PDF
    retriever = vectorstores[predicted_label].as_retriever(search_type="similarity", search_kwargs={"k": 5})
    relevant_docs = retriever.invoke(part)
    context_snippets = "\n".join([doc.page_content for doc in relevant_docs])

    # Feed into LLM prompt
    prompt_value = prompt.format(
        spare_parts=parts_list,
        failures=failure_list,
        KXE_brochure=context_snippets if "KXE_brochure" == predicted_label else "",
        XE_brochure=context_snippets if "XE_brochure" == predicted_label else "",
        XE_specs=context_snippets if "XE_specs" == predicted_label else "",
        original_critical_spares_list="\n".join(critical_list),
        risk_assessment=context_snippets if "risk_assessment" == predicted_label else "",

    )

    response = llm.invoke(prompt_value)

    critical_spares_list.append({
        "Critical Spare Parts List": response
    })

# Save results
critical_spares_list_df = pd.DataFrame(critical_spares_list)
output_path = "/content/drive/MyDrive/Colab Notebooks/critical_spares_list.csv"
critical_spares_list_df.to_csv(output_path, index=False)

In [ ]:
## FORMATTING RESULTS ##
# Load text
with open(output_path, "r", encoding="utf-8") as f:
    text = f.read()

# Normalize newlines (replace literal \n with actual newline)
text = text.replace("\\n", "\n")

# Split by numbered entries (more flexible)
entries = re.split(r"\n\d+\.\s+", text)
entries = [e.strip() for e in entries if e.strip()]

data = []

keys = ["CAT Part", "SAP Material Description", "U801 MRP",
        "Unit Cost (USD)", "Lead Time (days)", "Assumed Likelihood Score",
        "Estimated Severity Score", "Criticality Score", "Comments"]

for entry in entries:
    entry_dict = {}

    # Remove extra sections
    entry = re.split(r"Verification checks:|Notes and Verification Steps:|Assumptions and verification", entry)[0]

    # Clean up formatting
    entry = re.sub(r"[•\u2003\u2022─]+", " ", entry)
    entry = re.sub(r"\\n\\u2003|\\u2003\\u2003|\\n\\n", " ", entry)
    entry = re.sub(r"\\u2003", "", entry)
    entry = re.sub(r"\[.*?\]", "", entry)
    entry = re.sub(r"\s+", " ", entry).strip()

    # Extract key-value pairs
    for i, key in enumerate(keys):
        if i < len(keys) - 1:
            next_key = keys[i+1]
            pattern = f"{re.escape(key)}: (.*?)(?={re.escape(next_key)}:|$)"
        else:
            pattern = f"{re.escape(key)}: (.*)"
        match = re.search(pattern, entry)
        entry_dict[key] = match.group(1).strip() if match else ""

    data.append(entry_dict)

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV
output_path = "/content/drive/MyDrive/Colab Notebooks/formatted_critical_list.csv"
df.to_csv(output_path, index=False)

print(df)